In [1]:
import pyodbc
import pandas as pd
import decimal

In [2]:
string = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+
          r'DBQ=C:\Users\Edoardo\Documents\repositories\sorting-problem\DbOrdinamento.accdb')
conn = pyodbc.connect(string)
cursor = conn.cursor()
print([t[2] for t in cursor.tables()])
conn.close()

['_NS_LOCALE_STAMPA', '_NS_LOCALE_STAMPA1', 'MSysAccessStorage', 'MSysAccessXML', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', '_NS_LOCALE_STAMPA2', 'Copia di VariabiliMacchina', 'Errori di incollamento', 'Variabili', 'VariabiliMacchina', 'Q_ControllaOrdinato', 'Q_EstraiPerOrdinamento']


In [55]:
def read_data_from_access(path, table, query=None):
    string = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+
              'DBQ=' + path)
    conn = pyodbc.connect(string)
    cursor = conn.cursor()

    if query is None:
        query = f'select * from {table}'  # Q_EstraiPerOrdinamento, Variabili
    cursor.execute(query)

    data = cursor.fetchall()
    conn.close()

    columns = [c[0] for c in cursor.description]
    types = [c[1] for c in cursor.description]

    df = pd.DataFrame([list(row) for row in data], columns=columns)
    for i in range(len(columns)):
        if types[i] in [decimal.Decimal, int]:
            df[columns[i]] = df[columns[i]].astype(float)
    return df

In [75]:
path = r'C:\Users\Edoardo\Documents\repositories\sorting-problem\DbOrdinamento.accdb'
table = 'Q_EstraiPerOrdinamento'
query = "select * from Q_EstraiPerOrdinamento where (Macchina = 'Diamond' and [_C_for]=30.0 and NumeroCom=104912.0 and S='CE5917')"
df = read_data_from_access(path, 'Q_EstraiPerOrdinamento')


In [76]:
df.loc[df.Macchina=='Diamond'].sort_values('Prog')

,Macchina,Prog,_C_for,Var_K,NumeroCom,IDRIGA,Fascia,NrCol,QTS,S,T,DescCol01,DescCol02,DescCol03,DescCol04,DescCol05,DescCol06,DescCol07,DescCol08
2,Diamond,12.0,40.0,KB75,104942.0,1.0,0330,2,S,8N036,BI2633,##Blu,##Azzurro,None,None,None,None,None,None
3,Diamond,13.0,80.0,KB75,104979.0,1.0,0327,4,S,IA1921,IA2330,@@Campione Blu,@@Campione Blu,@@Campione Azzurro,@@Campione Arancione,None,None,None,None
4,Diamond,14.0,120.0,KB75,104915.0,1.0,0340,6,Q,IA0338,BI2324,process black U,process magenta U,process yellows U,process cyan U,@@Campione Azzurro,@@Campione Blu,None,None
5,Diamond,15.0,280.0,MPA0,104710.0,1.0,0397,8,Q,IB6678,IB6753,process yellows C,process magenta C,process cyan C,process black C,Rosso Cartella Colori XC001(logo Francine camp...,@@Campione Grigio,SIMIL ORO,7610 C (Marrone)
6,Diamond,16.0,160.0,MP85,104709.0,1.0,0337,8,Q,IB4978,IB6672,process yellows C,process magenta C,process cyan C,process black C,Rosso Cartella Colori XC001(logo Francine camp...,1817 C (Marrone),151 C (Arancio),process magenta C
7,Diamond,17.0,160.0,MP85,104708.0,1.0,0337,8,Q,IB6624,IB6624,process yellows C,process magenta C,process cyan C,process black C,Rosso Cartella Colori XC001(logo Francine camp...,7610 C (Marrone),@@Campione Rosa,XC014 Grigio fondo Francine pant. 428C
8,Diamond,18.0,1.0,SA90,26302.0,12.0,0337,0,S,YYIMPE,YYIMPE,None,None,None,None,None,None,None,None
9,Diamond,19.0,20.0,KA90,104881.0,1.0,0447,1,S,DA1488,IC1834,##Nero,None,None,None,None,None,None,None
12,Diamond,20.0,140.0,DXD5,104885.0,1.0,1000,4,S,BC0089,BC0089,485 C (Rosso),348 C (Verde),cool gray 6 C (Grigio),##Nero,None,None,None,None
13,Diamond,21.0,0.0,DXD5,104885.0,2.0,1000,4,S,BC0089,BC0089,485 C (Rosso),348 C (Verde),cool gray 6 C (Grigio),##Nero,None,None,None,None


In [48]:
df.head()

,Macchina,Prog,_C_for,Var_K,NumeroCom,IDRIGA,Fascia,NrCol,QTS,S,T,DescCol01,DescCol02,DescCol03,DescCol04,DescCol05,DescCol06,DescCol07,DescCol08
0,Onix108,10.0,20.0,AB90,104967.0,1.0,0910,1,S,EC1245,EE1817,##Nero,None,None,None,None,None,None,None
1,OnixK0,10.0,180.0,KB90,104750.0,1.0,0347,8,Q,IB5256,IB6406,process black U,##Nero,process magenta U,process yellows U,412 U (Grigio),7531 U (Grigio),180 U (Bordeaux),process cyan U
2,Diamond,12.0,40.0,KB75,104942.0,1.0,0330,2,S,8N036,BI2633,##Blu,##Azzurro,None,None,None,None,None,None
3,Diamond,13.0,80.0,KB75,104979.0,1.0,0327,4,S,IA1921,IA2330,@@Campione Blu,@@Campione Blu,@@Campione Azzurro,@@Campione Arancione,None,None,None,None
4,Diamond,14.0,120.0,KB75,104915.0,1.0,0340,6,Q,IA0338,BI2324,process black U,process magenta U,process yellows U,process cyan U,@@Campione Azzurro,@@Campione Blu,None,None


In [74]:
df.Macchina.unique()

array(['Diamond', 'OnixK0', 'Onix108'], dtype=object)

In [58]:
string = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+
          r'DBQ=C:\Users\Edoardo\Documents\repositories\sorting-problem\DbOrdinamento.accdb')
conn = pyodbc.connect(string)
cursor = conn.cursor()
cursor.execute('Select * from Q_EstraiPerOrdinamento')

new_cols = []
for col in cursor.description:
    column_name = col[0]
    column_type = col[1]
    if column_type == str:
        column_type = 'varchar'
    elif column_type == int:
        column_type = 'integer'
    elif column_type == decimal.Decimal:
        column_type = 'double'
    new_cols.append(column_name + ' ' + column_type)


In [61]:
query = f"CREATE TABLE Test_table({', '.join(new_cols)})"
cursor.execute(query)
conn.commit()
conn.close()

In [77]:
string = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+
          r'DBQ=C:\Users\Edoardo\Documents\repositories\sorting-problem\DbOrdinamento.accdb')
conn = pyodbc.connect(string)
cursor = conn.cursor() 
query1 = "update Q_EstraiPerOrdinamento set prog=10 where macchina='Diamond' and prog=10.0"
cursor.execute(query1)
conn.commit()
conn.close()